<!-- SPDX-License-Identifier: CC-BY-4.0 -->
<!-- Copyright Contributors to the ODPi Egeria project 2024. -->

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Surveying and Cataloguing Unity Catalog (UC)

## Introduction

Both Unity Catalog and Egeria are open source projects with the LF AI and Data.  The difference between these technologies is:

 * Unity Catalog is responsible for governing access to data; whereas Egeria governs the exchange of metadata between tools and systems, such as Unity Catalog.

 * Similarly, Unity Catalog maintains a metadata repository describing the data it is protecting.  In contrast, Egeria maintains a distributed network of metadata repositories containing metadata about the technology (systems, tools, data), the processes that are operating on them, along with the people and organizations involved.

This notebook shows the integration between Egeria and Unity Catalog. Egeria can run surveys of the catalogs within a Unity Catalog server, it can catalog the contents of these catalogs (so that they can be incorporated in larger governance processes and searches) and it is possible to provision catalogs, schemas and volumes to a Unity Catalog Server.

Run the code below to initialize Egeria's python libaries and create a client to the Egeria servers.

---


In [1]:
# Initialize pyegeria

%run ../../pyegeria/initialize-pyegeria.ipynb


In [2]:

egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)
token = egeria_tech.create_egeria_bearer_token()


---

## Loading support for Unity Catalog

The definition of the Unity Catalog connectors, templates and associated reference data are loaded via a [Content Pack](https://egeria-project.org/content-packs/) called `UnityCatalogContentPack.omarchive`.  The content pack can be loaded multiple times without ill-effect so run the following command to make sure it is loaded.

---

In [3]:

egeria_tech.add_archive_file("content-packs/UnityCatalogContentPack.omarchive", None, "qs-metadata-store")

print("Archive loaded!")


Archive loaded!


----

Run the command command below to confirm that the connectors that synchronize metadata between Egeria and Unity Catalog are running, waiting for work:

* UnityCatalogServerSynchronizer synchronizes catalog information from a Unity Catalog Server. It passes details of the catalogs it finds onto UnityCatalogInsideCatalogSynchronizer.
* UnityCatalogInsideCatalogSynchronizer synchronizes the schema, volume, table and function metadata between Egeria and Unity Catalog.

----

In [4]:
display_integration_daemon_status(['UnityCatalogServerSynchronizer', 'UnityCatalogInsideCatalogSynchronizer'], paging=True, width=150)

                                                 Integration Daemon Status @ Wed Jan 29 16:31:49 2025                                                 
╭───────────────────────────────────────┬────────┬─────────────────────┬────────┬────────────────────────────────────────────────┬───────────────────╮
│                                       │        │                     │ Min    │                                                │                   │
│                                       │        │                     │ Refre… │                                                │                   │
│ Connector Name                        │ Status │ Last Refresh Time   │ (mins) │ Target Element                                 │ Exception Message │
├───────────────────────────────────────┼────────┼─────────────────────┼────────┼────────────────────────────────────────────────┼───────────────────┤
│ UnityCatalogInsideCatalogSynchronizer │ WAITI… │ 2025-01-29T16:14:30 │ 60     │ ┏━━┳━━┳━━━━━

----

## Gathering data about your Unity Catalog Server

You need to provide Egeria with some basic information about your Unity Catalog server.  Fill in the details below.

-----

In [5]:
# This is the technology type if the server is the Open Source version of Unity Catalog.
# Use "Databricks Unity Catalog Server" if this is a Databricks cloud service version of Unity Catalog.
serverTechnologyType = "Unity Catalog Server"

# This is the network location used to call the Unity Catalog Server.
# For Unity Catalog running on your local machine, use:
#   * "http://localhost" if Egeria is also running natively on your local machine, or
#   * "http://host.docker.internal" if Egeria is running in a docker image.
#
hostURL="http://host.docker.internal"
portNumber="8087"

# This is a unique name that the server is known as
serverName="Unity Catalog 1"

# Add a short description of the server
description="Local instance of the Unity Catalog (UC) Server."

# This is the verion of Unity Catalog
versionIdentifier="v0.3.0-SNAPSHOT"

# This is the userId that the connectors will use when creating metadata in Egeria that describes resources in Unity Catalog.
serverUserId="uc1"

# These values are user in later sections
serverQualifiedName=serverTechnologyType + ":" + serverName
serverNetworkAddress=hostURL + ":" + portNumber
search_results = egeria_tech.find_in_asset_domain("coco-sus")

-----

## Survey a Unity Catalog Server

The Unity Catalog support includes the ability to survey the contents of a Unity Catalog Server.  This command creates a description of the Unity Catalog Server and runs a survey to understand its contents.  A summary of the survey results can be found in /distribution-hub/surveys.

---

In [8]:

createAndSurveyServerName="UnityCatalogServer:CreateAndSurveyGovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(createAndSurveyServerName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


In [9]:

requestParameters = {
    "hostURL" : hostURL,
    "portNumber" : portNumber,
    "serverName" : serverName,  
    "versionIdentifier" : versionIdentifier,
    "description" : description,
    "serverUserId" : serverUserId
}

egeria_tech.initiate_gov_action_process(createAndSurveyServerName, None, None, None, requestParameters, None, None)


'87cd2706-75c3-43dc-8415-7919073eb3ea'

----

The command below displays the latest governance actions.  You should see they are in **ACTIONED** status.  If you see failures it means that either the Unity Catalog server is not running or the values describing its location are not correct.  If Unity Catalog is down, restart it and re-run the cell above.  If you realize one or more of the values describing Unity Catalog is not right, go to the bottom of this notebook to the section **Starting again ...** and run the delete process.  Then you can go back up to the section called **Gathering data about your Unity Catalog Server** and carry on from there.

----

In [11]:
display_engine_activity_c()

                                                     Engine Action Status for Platform https://host.docker.internal:9443 @ Tue Jan 28 21:28:11 2025                                                     
╭─────────────────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────┬───────────────┬─────────────────────┬──────────────────────────────────────────────╮
│ Requested Time      │ Core Info                                   │ Target Elements                             │ Action Status │ Completion Time     │ Core Results                                 │
├─────────────────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────────┼─────────────────────┼──────────────────────────────────────────────┤
│ 2025-01-28T21:28:00 │                                             │                                             │ ACTIONED      │ 2025-01-28T21:28:06 │                                           

----

Open up the survey file and review the contents of the Unity Catalog Server. Notice there can be multiple catalogs in a Unity Catalog Server.  Also notice the hierarchical naming of the unity catalog elements.  Catalogs have schemas inside them and the schemas can have tables, functions and/or volumes within them.

----

Use the command `hey_egeria_ops show engines activity --compressed` to view the governance actions that ran as a result of the survey requests.  
There were two steps.  First it created a `SoftwareServer` element to represent the Unity Catalog Server. 
This stores the network address of the server.  Then the survey was run using this element.

-----

Now navigate to `distribution-hub/logs/openlineage/GovernanceActions`.  This directory stores the open lineage events created when the surveys were run.  Each event record the start or stop of a governance action.



---

## Cataloguing Unity Catalog metadata in Egeria

If the surveys look interesting, it is possible to synchronize the metadata between Unity Catalog and Egeria.  Run the command command below. 

The process shown below will request that the contents of the first Unity Catalog server is catalogued into Egeria.

---

In [12]:

createAndCatalogServerName="UnityCatalogServer:CreateAsCatalogTargetGovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(createAndCatalogServerName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


----

The code below will configure these connectors to catalog the server.

----

In [13]:

requestParameters = {
    "hostURL" : hostURL,
    "portNumber" : portNumber,
    "serverName" : serverName,  
    "versionIdentifier" : versionIdentifier,
    "description" : description,
    "serverUserId" : serverUserId
}

egeria_tech.initiate_gov_action_process(createAndCatalogServerName, None, None, None, requestParameters, None, None)


'93108d89-e8aa-4d46-9a59-52abdf7fa602'

----

Again it is possible to watch the execution of the process.  Notice that only 2 engine actions are activated.

----

In [14]:
display_engine_activity_c()

                                                     Engine Action Status for Platform https://host.docker.internal:9443 @ Tue Jan 28 21:30:10 2025                                                     
╭─────────────────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────┬───────────────┬─────────────────────┬──────────────────────────────────────────────╮
│ Requested Time      │ Core Info                                   │ Target Elements                             │ Action Status │ Completion Time     │ Core Results                                 │
├─────────────────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────────┼─────────────────────┼──────────────────────────────────────────────┤
│ 2025-01-28T21:28:40 │                                             │                                             │ ACTIONED      │ 2025-01-28T21:28:43 │                                           

----

The effect of the process is to configure the *UnityCatalogServerSynchronizer* connector to extract the catalog found in the Unity Catalog Server and then configure the *UnityCatalogInsideCatalogSynchronizer* to catalog each one it finds.
The Target Element column shows the details of the server/catalogs they are working with.

----

In [15]:
display_integration_daemon_status(['UnityCatalogServerSynchronizer', 'UnityCatalogInsideCatalogSynchronizer'], paging=True, width=200)

                                                                          Integration Daemon Status @ Tue Jan 28 21:30:17 2025                                                                          
╭───────────────────────────────────────┬────────┬─────────────────────┬────────┬──────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────╮
│                                       │        │                     │ Min    │                                                                                                  │                   │
│                                       │        │                     │ Refre… │                                                                                                  │                   │
│ Connector Name                        │ Status │ Last Refresh Time   │ (mins) │ Target Element                                                                                   │ Exception Messa

---

Below is a graph of the elements found during the cataloguing process.  You can see the schemas and the tables, volumes and functions underneath them.  The types of the elements are shown using [open metadata types](https://egeria-project.org/connectors/unity-catalog/#open-metadata-type-mapping-for-unity-catalog).

---

In [50]:
unity_default_qualified_name="Unity Catalog Schema:" + serverNetworkAddress + ":unity.default"

guid=egeria_tech.get_element_guid_by_unique_name(unity_default_qualified_name)

print(guid)

graph = egeria_tech.get_asset_lineage_mermaid_graph(guid)
icon = 'n3@{ img: "https://raw.githubusercontent.com/odpi/egeria/refs/heads/main/assets/img/ODPi_Egeria_Logo_color.png", h: 200, w: 300, pos: "b"'
graph = graph 
ln = '\nclick 83621335-f39d-4710-ab4c-bb9cc1f84b1d "https://egeria-project.org/"'
print(graph)
render_mermaid(graph+ln)

83621335-f39d-4710-ab4c-bb9cc1f84b1d
---
title: Lineage Graph for Asset - default [83621335-f39d-4710-ab4c-bb9cc1f84b1d]
---
flowchart TD
%%{init: {"flowchart": {"htmlLabels": false}} }%%

83621335-f39d-4710-ab4c-bb9cc1f84b1d("`*DeployedDatabaseSchema*
**default**`")
daf8bb5f-417f-4fa7-837e-0fed7d145de3("`*VirtualRelationalTable*
**marksheet**`")
570e8558-a2e8-44f4-b0a6-9460eb0f0712("`*DataFolder*
**unity.default.marksheet_storageLocation**`")
f83148ea-a3c9-4738-b24c-903c4c110abc("`*DeployedAPI*
**sum**`")
ea42e044-9073-466f-9cd8-2d19b8ef186a("`*DataFolder*
**json_files**`")
9c55271a-be19-4d24-b46f-465573b58cac("`*VirtualRelationalTable*
**marksheet_uniform**`")
d3b53bda-d95a-4b9e-a829-40768dd8259c("`*DataFolder*
**unity.default.marksheet_uniform_storageLocation**`")
c206a44f-1b2e-424c-9657-f94d35bbdf36("`*DeployedAPI*
**lowercase**`")
1426c8a6-5cf6-4cb7-8674-51e1dbcd85cf("`*VirtualRelationalTable*
**numbers**`")
f2569e3f-2b26-415b-a656-4097fd502e3d("`*DataFolder*
**unity.default.numbe

----

You can use the following commands to show the elements from Unity Catalog in Egeria:

* `hey_egeria cat show tech-type-elements --tech_type 'Unity Catalog Server'` for the Unity Catalog Servers.
* `hey_egeria cat show tech-type-elements --tech_type 'Unity Catalog Catalog'` for the Unity Catalog Catalogs.
* `hey_egeria cat show tech-type-elements --tech_type 'Unity Catalog Schema'` for the Unity Catalog Schemas.
* `hey_egeria cat show tech-type-elements --tech_type 'Unity Catalog Table'` for the Unity Catalog Tables.
* `hey_egeria cat show tech-type-elements --tech_type 'Unity Catalog Volume'` for the Unity Catalog Volumes.
* `hey_egeria cat show tech-type-elements --tech_type 'Unity Catalog Function'` for the Unity Catalog Functions.

You can also use `hey_egeria_cat show assets inventory` to search for assets that include the word `inventory` in it.

----

It is also possible to use python functions to retrieve information about the Unity Catalog resources.  This uses the `egeria_cat` client.

----

In [17]:
from pyegeria.commands.ops.list_catalog_targets import display_catalog_targets

display_catalog_targets("UnityCatalogInsideCatalogSynchronizer",
                       view_server,
                       url,
                       user_id,
                       user_pwd)


                                             All Catalog Targets for Integration Connector UnityCatalogInsideCatalogSynchronizer @ Tue Jan 28 21:30:28 2025                                             
╭──────────────────────────────────────┬─────────────────────────────────────────────┬────────────────────────────────────────────┬─────────────────────────────────────────────┬──────────────────────╮
│                                      │                                             │                                            │                                             │ Operational          │
│ Name of Target                       │ Catalog Target                              │ Configuration Properties                   │ Template Properties                         │ Instructions         │
├──────────────────────────────────────┼─────────────────────────────────────────────┼────────────────────────────────────────────┼─────────────────────────────────────────────┼───────────────────

In [18]:

egeria_cat = EgeriaCat(view_server, url, user_id, user_pwd)
egeria_cat.set_bearer_token(token)


----

The command below retrieves assets that match the search query.  If you see the message "no assets found" then re-run the request as the synchronization process may not have been comopleted.

----

In [19]:

search_results = egeria_cat.find_in_asset_domain("numbers")

print_search_results(search_results)
       


Asset: 
 > DeployedDatabaseSchema [83621335-f39d-4710-ab4c-bb9cc1f84b1d]
 > class: AssetProperties
 > typeName: DeployedDatabaseSchema
 > qualifiedName: Unity Catalog Schema:http://host.docker.internal:8087:unity.default
 > displayName: default
 > displayDescription: Default schema
 > name: default
 > resourceName: unity.default
 > resourceDescription: Default schema
 > deployedImplementationType: Unity Catalog Schema
Matching Element:
   > DeployedAPI [f83148ea-a3c9-4738-b24c-903c4c110abc]
   > qualifiedName: Unity Catalog Function:http://host.docker.internal:8087:unity.default.sum
   > name: sum
   > description: Adds two numbers.
   > resourceName: unity.default.sum
   > deployedImplementationType: Unity Catalog Function
Matching Element:
   > VirtualRelationalTable [1426c8a6-5cf6-4cb7-8674-51e1dbcd85cf]
   > qualifiedName: Unity Catalog Table:http://host.docker.internal:8087:unity.default.numbers
   > name: numbers
   > description: External table
   > resourceName: unity.default.

----
This shows that metadata can be copied from Unity Catalog into Egeria's metadata repository and represented using Open Metadata Types.  The next part of the demonstration show metadata flowing from Egeria to Unity Catalog.


----

# Provisioning Unity Catalog (UC) from Egeria

Egeria has the ability to provision resources into Unity Catalog.  The desired resources are described in Egeria using Open Metadata Elements and linked to the representation of the server, catalog or schema where the new resource is to be created.  Once the description is in place, the appropriate Unity Catalog Integration Connector will create a matching definition in Unity Catalog.

The sections below go through creating a catalog, then a schema within that catalog and then a volume within that schema.  Notice that the ordering is important.  The catalog must be created before it schemas etc.

-----

----

## Provision a new catalog into a Unity Catalog (UC) Server

There is only one step to provision a new catalog into a Unity Catalog Server:

-----

In [20]:

provisionCatalogName="Provision:UnityCatalogCatalog:GovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(provisionCatalogName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


----

Fill in details about the new catalog that you want to create.

----

In [22]:

catalogName = "new_catalog"
catalogDescription = "My new catalog."


----

Now run the governance process to create the catalog.

----

In [23]:

requestParameters = {
    "ucServerQualifiedName" : serverQualifiedName,
    "serverNetworkAddress" : serverNetworkAddress,
    "ucCatalogName" : catalogName,
    "versionIdentifier" : versionIdentifier,
    "description" : catalogDescription
}

egeria_tech.initiate_gov_action_process(provisionCatalogName, None, None, None, requestParameters, None, None)


'88a87432-5729-4e00-89fd-782bda3d26a6'

------

The code below shows the catalog you now have defined.

------

In [24]:

catalog_qualified_name="Unity Catalog Catalog:" + serverNetworkAddress + ":" + catalogName

element=egeria_tech.get_element_by_unique_name(catalog_qualified_name)

print_element("", element)



DataAccessManager [c08f96a2-e247-4e17-8014-f8e3619e5890]
 > qualifiedName: Unity Catalog Catalog:http://host.docker.internal:8087:new_catalog
 > name: new_catalog
 > description: My new catalog.
 > deployedImplementationType: Unity Catalog Catalog


----

The next command lists the relationships to other elements that this catalog has:

----

In [25]:

catalog_guid=egeria_tech.get_element_guid_by_unique_name(catalog_qualified_name)
print(catalog_guid)

related_elements = egeria_tech.get_related_elements(catalog_guid)

print_related_elements("", related_elements)


c08f96a2-e247-4e17-8014-f8e3619e5890

ExternalIdScope [25f0f1e3-4c66-4e2a-8a83-5b00e8911c05]
 > permittedSynchronization: FromThirdParty

   ExternalId [1e78a65a-b44d-4733-8533-fc222157b383]
    > keyPattern: Local Key
    > externalInstanceLastUpdateTime: 1737827629629
    > identifier: f97d089d-dc99-46ef-85cb-50a8e93b2cff
    > externalInstanceVersion: 0
    > externalInstanceCreationTime: 1737827629629
    > qualifiedName: f97d089d-dc99-46ef-85cb-50a8e93b2cff:abd0aeb3-7ce0-426d-bcb8-723cb75d3276
    > externalInstanceTypeName: schema

ExternalIdScope [96373b60-f305-4703-b3e3-6cdc27fdb0b8]
 > permittedSynchronization: FromThirdParty

   ExternalId [df4a363e-0ddf-4cfd-91c3-86a9885464e8]
    > keyPattern: Local Key
    > externalInstanceLastUpdateTime: 1737827583740
    > identifier: d694cfa0-dbaf-4059-8e04-2b8ba5da96fe
    > externalInstanceVersion: 0
    > externalInstanceCreationTime: 1737827583740
    > qualifiedName: d694cfa0-dbaf-4059-8e04-2b8ba5da96fe:0d4e6237-89ca-4f5e-8787-7a6

----

## Provision a new schema into a Unity Catalog (UC) Catalog

This is the process to provision a new schema into a Unity Catalog Catalog:

-----

In [26]:

provisionSchemaName="Provision:UnityCatalogSchema:GovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(provisionSchemaName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


----

Fill in details about the new schema that you want to create.

----

In [27]:

schemaName = "new_schema"
schemaDescription = "My new schema."


----

Now run the governance process to create the schema.

----

In [28]:

requestParameters = {
    "serverNetworkAddress" : serverNetworkAddress,
    "ucCatalogName" : catalogName,
    "ucSchemaName" : schemaName,
    "versionIdentifier" : versionIdentifier,
    "description" : schemaDescription
}

egeria_tech.initiate_gov_action_process(provisionSchemaName, None, None, None, requestParameters, None, None)


'8dba48d1-b96e-4e67-8522-3daf14dbf20a'

------

The code below shows the schema you now have defined.

------

In [29]:

schema_qualified_name="Unity Catalog Schema:" + serverNetworkAddress + ":" + catalogName + "." + schemaName

element=egeria_tech.get_element_by_unique_name(schema_qualified_name)

print_element("", element)



DeployedDatabaseSchema [990dbfc3-e015-47c8-8fef-503f3ac1382f]
 > qualifiedName: Unity Catalog Schema:http://host.docker.internal:8087:new_catalog.new_schema
 > name: new_schema
 > description: My new schema.
 > resourceName: new_catalog.new_schema
 > deployedImplementationType: Unity Catalog Schema


----

And the elements related to it ...

____

In [30]:

schema_guid=egeria_tech.get_element_guid_by_unique_name(schema_qualified_name)
print(schema_guid)

related_elements = egeria_tech.get_related_elements(schema_guid)

print_related_elements("", related_elements)


990dbfc3-e015-47c8-8fef-503f3ac1382f

ExternalIdLink [7a771536-8461-46de-be0d-6933203de23f]
 > mappingProperties: {ucCatalogName=new_catalog, ucSchemaName=new_schema, serverNetworkAddress=http://host.docker.internal:8087, schemaName=new_schema}
 > lastSynchronized: 1738099739073
 > source: Unity Catalog Catalog

   ExternalId [1e78a65a-b44d-4733-8533-fc222157b383]
    > keyPattern: Local Key
    > externalInstanceLastUpdateTime: 1737827629629
    > identifier: f97d089d-dc99-46ef-85cb-50a8e93b2cff
    > externalInstanceVersion: 0
    > externalInstanceCreationTime: 1737827629629
    > qualifiedName: f97d089d-dc99-46ef-85cb-50a8e93b2cff:abd0aeb3-7ce0-426d-bcb8-723cb75d3276
    > externalInstanceTypeName: schema

ServerAssetUse [a21cdbfc-7bbe-4b20-8501-8273f380594c]
 > useType: Owns

   DataAccessManager [c08f96a2-e247-4e17-8014-f8e3619e5890]
    > qualifiedName: Unity Catalog Catalog:http://host.docker.internal:8087:new_catalog
    > name: new_catalog
    > description: My new catalog.
 

----

## Provision a new volume into a Unity Catalog (UC) Schema

This is the process to provision a new volume into a Unity Catalog Schema:

-----

In [31]:

provisionVolumeName="Provision:UnityCatalogVolume:GovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(provisionVolumeName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


----

Fill in details about the new volume that you want to create.

----

In [32]:

volumeName = "new_volume"
volumeDescription = "My new volume."
storageLocation = "data/new_volume"
volumeType = "EXTERNAL"


----

Now run the governance process to create the volume.

----

In [33]:

requestParameters = {
    "serverNetworkAddress" : serverNetworkAddress,
    "ucCatalogName" : catalogName,
    "ucSchemaName" : schemaName,
    "ucVolumeName" : volumeName,
    "versionIdentifier" : versionIdentifier,
    "ucStorageLocation" : storageLocation,
    "description" : volumeDescription,
    "ucVolumeType" : volumeType
}

egeria_tech.initiate_gov_action_process(provisionVolumeName, None, None, None, requestParameters, None, None)


'0494ab54-5481-4dc7-9551-23aaa6b2f1bb'

------

The code below shows the volume you now have defined.

------

In [36]:

volume_qualified_name="Unity Catalog Volume:" + serverNetworkAddress + ":" + catalogName + "." + schemaName + "." + volumeName

element=egeria_tech.get_element_by_unique_name(volume_qualified_name)

print_element("", element)



DataFolder [61708091-fe45-42c2-975d-edbef4626df9]
 > pathName: data/new_volume
 > versionIdentifier: v0.3.0-SNAPSHOT
 > qualifiedName: Unity Catalog Volume:http://host.docker.internal:8087:new_catalog.new_schema.new_volume
 > name: new_volume
 > description: My new volume.
 > resourceName: new_catalog.new_schema.new_volume
 > deployedImplementationType: Unity Catalog Volume


----

And the elements related to this schema ...

----

In [37]:

volume_guid=egeria_tech.get_element_guid_by_unique_name(volume_qualified_name)
print(volume_guid)

related_elements = egeria_tech.get_related_elements(volume_guid)

print_related_elements("", related_elements)


61708091-fe45-42c2-975d-edbef4626df9

ExternalIdLink [ee422263-70e0-4c47-bcea-f1bea4b27866]
 > mappingProperties: {ucCatalogName=new_catalog, ucVolumeName=new_volume, ucSchemaName=new_schema, serverNetworkAddress=http://host.docker.internal:8087}
 > lastSynchronized: 1738099993127
 > source: Unity Catalog Catalog

   ExternalId [4645d2cc-2a05-452a-abaa-20a6712a04d3]
    > keyPattern: Local Key
    > externalInstanceLastUpdateTime: 1738099993050
    > identifier: 986e5246-6cb7-4171-a5d9-79b3e10b1d2b
    > externalInstanceVersion: 0
    > externalInstanceCreationTime: 1738099993050
    > qualifiedName: 986e5246-6cb7-4171-a5d9-79b3e10b1d2b:10de4ed6-4653-497f-bc08-b47cb8eeb6a0
    > externalInstanceTypeName: volume

ReferenceableFacet [0e05bc97-32e9-419b-83e8-37a676632877]
 > source: Unity Catalog (UC)

   PropertyFacet [7322b3cb-1e8a-476e-b011-09e7274bcc47]
    > schemaVersion: 1.0
    > qualifiedName: Unity Catalog Volume:http://host.docker.internal:8087:new_catalog.new_schema.new_volume

----

# Reviewing the integration

As Egeria is exchanging messages with Unity Catalog, it is building a map of the identifiers from Unity Catalog and mapping them to the elements that have been created in the open metadata ecosystem.

The functions below retrieve the mappings for each catalog within Unity Catalog (UC) serevers known to Egeria.

-----

In [38]:

unity_catalog_catalogs = egeria_tech.get_technology_type_elements("Unity Catalog Catalog")

if unity_catalog_catalogs:
    for catalog in unity_catalog_catalogs:
        print()
        print("----------------------------")
        print_external_id_map(catalog)
        


----------------------------

DataAccessManager [d7ce69ff-62d7-4209-9dc3-c9c9da583c8e]
 > class: ReferenceableProperties
 > typeName: DataAccessManager
 > name: unity
 > description: Main catalog
 > deployedImplementationType: Unity Catalog Catalog
 > qualifiedName: Unity Catalog Catalog:http://host.docker.internal:8087:unity

 > Permitted Synchronization: FromThirdParty

      ExternalIdLink [328d2272-5583-4676-9ca7-c6d574a91ab4]
       > lastSynchronized: 1738099730664
       > mappingProperties: {ucCatalogName=unity, ucSchemaName=default, serverNetworkAddress=http://host.docker.internal:8087, schemaName=default}
       > source: Unity Catalog Catalog

         DeployedDatabaseSchema [83621335-f39d-4710-ab4c-bb9cc1f84b1d]
          > qualifiedName: Unity Catalog Schema:http://host.docker.internal:8087:unity.default
          > name: default
          > description: Default schema
          > resourceName: unity.default
          > deployedImplementationType: Unity Catalog Schema

 >

----

# Starting again ...

If you made a mistake in the server details supplied for Unity Catalog, it is possible to remove the server definition using the delete process as follows.  Leave all of the settings unchanged and run this process.  Then you can go to the top, change the settings and re-run the survey and/or catalog processes are desired.

-----

In [ ]:
deleteCatalogName="UnityCatalogServer:DeleteAssetWithTemplateGovernanceActionProcess"

process_guid = egeria_tech.get_element_guid_by_unique_name(deleteCatalogName)

process_graph = egeria_tech.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


In [ ]:
requestParameters = {
    "hostURL" : hostURL,
    "portNumber" : portNumber,
    "serverName" : serverName,  
    "versionIdentifier" : versionIdentifier,
    "description" : description,
    "serverUserId" : serverUserId
}

egeria_tech.initiate_gov_action_process(deleteCatalogName, None, None, None, requestParameters, None, None)

------